# Crosshatch

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joeljose/Crosshatch/blob/main/crosshatch.ipynb)

Turn portrait photos into crosshatch drawings using automatic segmentation and hatch pattern blending.

This notebook walks through the full pipeline step by step, then gives you a one-call function to process your own images.

## What is Crosshatching?

Crosshatching is the drawing of two layers of hatching at right-angles to create a mesh-like pattern. Multiple layers in varying directions can be used to create textures. Crosshatching is often used to create tonal effects, by varying the spacing of lines or by adding additional layers of lines. Crosshatching is used in pencil drawing, but is particularly useful with pen and ink drawing, to create the impression of areas of tone, since the pen can only create a solid black line.

## Visual Walkthrough

Let us look at the process of creating a crosshatch drawing.

![image](https://github.com/joeljose/Crosshatch/raw/main/assets/tutorial/land0.jpg)

First we draw the edges and contours we see in our photo:

![image](https://github.com/joeljose/Crosshatch/raw/main/assets/tutorial/land1.jpg)

In crosshatching, we create dark regions by drawing multiple hatches on those areas. Lighter areas contain progressively fewer hatches:

![image](https://github.com/joeljose/Crosshatch/raw/main/assets/tutorial/land2.jpg)

![image](https://github.com/joeljose/Crosshatch/raw/main/assets/tutorial/land3.jpg)

![image](https://github.com/joeljose/Crosshatch/raw/main/assets/tutorial/land3.5.jpg)

![image](https://github.com/joeljose/Crosshatch/raw/main/assets/tutorial/land3.7.jpg)

And the finished drawing:

![image](https://github.com/joeljose/Crosshatch/raw/main/assets/tutorial/land4.jpg)

## Our Approach

We imitate the steps above in code:

1. **Segment** — isolate the subject from the background using rembg
2. **Resize** — scale the image to match the hatch texture dimensions
3. **Layer** — place the subject on a white background
4. **Histogram** — analyze the tonal distribution of the image
5. **Threshold** — split the tonal range into three equal-area zones
6. **Hatch** — map a different line texture onto each zone
7. **Blend** — combine the hatch layers into the final drawing

---
## Install Dependencies

In [ ]:
import os

# Colab only downloads the notebook file, not the rest of the repo.
# Clone so we have the assets/ directory (textures, sample images, etc.)
if not os.path.exists('Crosshatch'):
    !git clone -q https://github.com/joeljose/Crosshatch.git
os.chdir('Crosshatch')

# Pin numpy to Colab's pre-installed version to avoid breaking other packages
import importlib.metadata as _md
_np_ver = _md.version('numpy')
!pip install -q "rembg[cpu]" "numpy=={_np_ver}"

---
## Import Libraries

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from rembg import new_session, remove

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

---
## Load Segmentation Model

[rembg](https://github.com/danielgatis/rembg) removes the background from images using the `u2net_human_seg` model, which is optimized for human subjects. The model is downloaded automatically on first use.

In [ ]:
session = new_session("u2net_human_seg")
print('Segmentation model loaded')

---
## Helper Functions

In [ ]:
def segment_person(image, session):
    """Segment the main subject using rembg."""
    if len(image.shape) == 2:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    pil_image = Image.fromarray(image_rgb)
    mask_pil = remove(pil_image, session=session, only_mask=True)
    mask = np.array(mask_pil)
    mask = np.where(mask > 128, 255, 0).astype(np.uint8)
    return mask


def calculate_thresholds(image):
    """Split the tonal range into three equal-area zones.

    Returns the pixel values at the 25th, 50th, and 75th percentile
    of the non-white pixels.
    """
    counts, _ = np.histogram(image, bins=256, range=(0, 256))
    total = np.sum(counts[:255])  # exclude white background

    thresholds = []
    cum_sum = 0
    targets = [total * p for p in (0.25, 0.50, 0.75)]

    for i in range(255):
        cum_sum += counts[i]
        if targets and cum_sum > targets[0]:
            thresholds.append(i)
            targets.pop(0)

    return tuple(thresholds)


def blend_images(images):
    """Blend a list of images with equal weight."""
    fraction = 1.0 / len(images)
    output = np.zeros_like(images[0], dtype=np.float32)
    for img in images:
        output += img.astype(np.float32) * fraction
    return output.astype(np.uint8)


def show_images(images, titles, figsize=(15, 5)):
    """Display images side by side."""
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img, cmap='gray' if len(img.shape) == 2 else None)
        ax.set_title(title)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

---
## Load Hatch Textures

We use four hand-drawn hatch patterns:

| Texture | Purpose |
|---|---|
| **right** | Diagonal lines (\\) — darkest tones |
| **left** | Diagonal lines (/) — mid tones |
| **horizontal** | Horizontal lines — lightest tones (classic style) |
| **vortex** | Circular swirl — lightest tones (artistic style) |

In [ ]:
left = cv2.imread('assets/textures/leftx.png', 0)
right = cv2.imread('assets/textures/rightx.png', 0)
horizontal = cv2.imread('assets/textures/horizontalx.png', 0)
vortex = cv2.imread('assets/textures/vortexx.png', 0)

print(f'Texture dimensions: {left.shape}')

show_images(
    [right, left, horizontal, vortex],
    ['Right (darkest)', 'Left (mid)', 'Horizontal (lightest)', 'Vortex (alt lightest)'],
    figsize=(20, 5),
)

---
## Load Sample Portrait

In [ ]:
image_path = 'assets/samples/lena.png'

image_color = cv2.imread(image_path)
image_gray = cv2.imread(image_path, 0)

print(f'Image size: {image_gray.shape}')
plt.imshow(cv2.cvtColor(image_color, cv2.COLOR_BGR2RGB))
plt.title('Original')
plt.axis('off')
plt.show()

---
## Step 1 — Segment the Subject

rembg automatically detects and segments the subject, returning a binary mask that separates the person from the background.

In [ ]:
mask = segment_person(image_color, session)

show_images(
    [cv2.cvtColor(image_color, cv2.COLOR_BGR2RGB), mask],
    ['Original', 'Segmentation Mask'],
    figsize=(12, 5),
)

## Step 2 — Resize

The hatch textures are 2100 px. We resize the portrait so that its largest side is 1200 px — this produces visually appealing hatch line density.

In [ ]:
max_unit = 1200
hatch_unit = 2100

height, width = image_gray.shape
ratio = max_unit / max(width, height)
new_width = int(ratio * width)
new_height = int(ratio * height)

image_resized = cv2.resize(image_gray, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)
mask_resized = cv2.resize(mask, (new_width, new_height), interpolation=cv2.INTER_LANCZOS4)

print(f'Resized to {new_width} x {new_height}')

## Step 3 — Layer on White Background

We place the segmented subject on a pure-white canvas. This ensures the background doesn't interfere with the hatching.

In [ ]:
background = np.ones_like(image_resized) * 255
layered = np.where(mask_resized == 255, image_resized, background)

plt.imshow(layered, cmap='gray')
plt.title('Layered (subject on white)')
plt.axis('off')
plt.show()

## Step 4 — Analyze Tonal Range

We plot the histogram of the layered image and find three threshold values that divide the non-white pixel area into equal thirds. Each third will receive a different hatch texture.

![histogram thresholds](https://github.com/joeljose/Crosshatch/raw/main/assets/tutorial/hist.png)

In [ ]:
# Plot the histogram
counts, bins = np.histogram(layered, range(257))
plt.bar(bins[:-1] - 0.5, counts, width=1, edgecolor='none')
plt.xlim([-0.5, 265.5])
plt.xlabel('Pixel intensity')
plt.ylabel('Count')
plt.title('Histogram of layered image')
plt.show()

# Calculate thresholds
thresh1, thresh2, thresh3 = calculate_thresholds(layered)
print(f'Thresholds: {thresh1}, {thresh2}, {thresh3}')

## Step 5 — Apply Hatch Patterns

Each hatch layer covers one tonal zone:

| Layer | Texture | Tonal zone |
|---|---|---|
| hatch 1 | right (\\) | darkest — pixels below threshold 1 |
| hatch 2 | left (/) | mid — pixels below threshold 2 |
| hatch 3 | horizontal or vortex | lightest — pixels below threshold 3 |

In [ ]:
# Crop textures to match the resized image
left_crop = left[:new_height, :new_width]
right_crop = right[:new_height, :new_width]
horizontal_crop = horizontal[:new_height, :new_width]

# Vortex needs a center crop
start_y = (hatch_unit - new_height) // 2
start_x = (hatch_unit - new_width) // 2
vortex_crop = vortex[start_y:start_y + new_height, start_x:start_x + new_width]

# Build hatch layers (using horizontal for the third layer)
hatch1 = np.where(layered < thresh1, right_crop, background)
hatch2 = np.where(layered < thresh2, left_crop, background)
hatch3 = np.where(layered < thresh3, horizontal_crop, background)

show_images(
    [hatch1, hatch2, hatch3],
    ['Hatch 1 (right — darkest)', 'Hatch 2 (left — mid)', 'Hatch 3 (horizontal — lightest)'],
    figsize=(18, 5),
)

## Step 6 — Blend

All three hatch layers are blended with equal weight to produce the final crosshatch drawing.

In [ ]:
output = blend_images([hatch1, hatch2, hatch3])

show_images(
    [cv2.cvtColor(image_color, cv2.COLOR_BGR2RGB), output],
    ['Original', 'Crosshatch (horizontal)'],
    figsize=(14, 6),
)

cv2.imwrite('output_horizontal.jpg', output)
print('Saved output_horizontal.jpg')

---
## Vortex Style

Swapping the third hatch layer from horizontal lines to a circular vortex pattern gives a more artistic look.

In [ ]:
hatch3_vortex = np.where(layered < thresh3, vortex_crop, background)
output_vortex = blend_images([hatch1, hatch2, hatch3_vortex])

show_images(
    [output, output_vortex],
    ['Horizontal style', 'Vortex style'],
    figsize=(14, 6),
)

cv2.imwrite('output_vortex.jpg', output_vortex)
print('Saved output_vortex.jpg')

---
## Complete Function

Everything above wrapped into a single reusable function.

In [ ]:
def create_crosshatch(image_path, output_path='output.jpg', hatch_style='horizontal',
                      max_dimension=1200):
    """Create a crosshatch drawing from a portrait image.

    Args:
        image_path: Path to the input image.
        output_path: Where to save the result.
        hatch_style: 'horizontal' or 'vortex'.
        max_dimension: Resize the longest side to this value.

    Returns:
        The crosshatched image as a numpy array.
    """
    hatch_unit = 2100

    # Load
    img_color = cv2.imread(image_path)
    img_gray = cv2.imread(image_path, 0)
    h, w = img_gray.shape

    # Segment
    mask = segment_person(img_color, session)

    # Resize
    r = max_dimension / max(w, h)
    nw, nh = int(r * w), int(r * h)
    img_resized = cv2.resize(img_gray, (nw, nh), interpolation=cv2.INTER_LANCZOS4)
    mask_resized = cv2.resize(mask, (nw, nh), interpolation=cv2.INTER_LANCZOS4)

    # Layer on white
    bg = np.ones_like(img_resized) * 255
    layered = np.where(mask_resized == 255, img_resized, bg)

    # Thresholds
    t1, t2, t3 = calculate_thresholds(layered)

    # Crop textures
    l_crop = left[:nh, :nw]
    r_crop = right[:nh, :nw]
    h_crop = horizontal[:nh, :nw]
    sy = (hatch_unit - nh) // 2
    sx = (hatch_unit - nw) // 2
    v_crop = vortex[sy:sy + nh, sx:sx + nw]

    # Hatch layers
    h1 = np.where(layered < t1, r_crop, bg)
    h2 = np.where(layered < t2, l_crop, bg)
    third = v_crop if hatch_style == 'vortex' else h_crop
    h3 = np.where(layered < t3, third, bg)

    # Blend and save
    result = blend_images([h1, h2, h3])
    cv2.imwrite(output_path, result)
    return result


print('create_crosshatch() defined')

---
## Try Your Own Image

1. Click the **folder icon** in the left sidebar to open the file browser
2. Navigate into the **Crosshatch** folder
3. **Upload** your portrait image there (drag and drop or right-click > Upload)
4. **Edit the filename** in the cell below to match your uploaded file
5. Run the cell

In [ ]:
uploaded_filename = 'my_portrait.jpg'  # <-- change this to your uploaded filename

result = create_crosshatch(
    uploaded_filename,
    'my_crosshatch.jpg',
    hatch_style='horizontal',  # change to 'vortex' for a different look
)

# Show original vs result
original_rgb = cv2.cvtColor(cv2.imread(uploaded_filename), cv2.COLOR_BGR2RGB)
show_images(
    [original_rgb, result],
    ['Original', 'Crosshatch'],
    figsize=(14, 6),
)

## Download Result

In [ ]:
from google.colab import files

files.download('my_crosshatch.jpg')

---
## Tips

- **Good contrast** — Images with clear lighting and contrast produce the best hatching.
- **Style choice** — Use `'horizontal'` for a classic look and `'vortex'` for something more artistic.
- **Resolution** — The `max_dimension` parameter controls output size. Larger values preserve more detail but take longer.